## Agents on Kubeflow 🤓

In this tutorial we will be training a reinforcement learning agent from the [tensorflow/agents](https://github.com/tensorflow/agents) project on Kubernetes using [Kubeflow](https://github.com/google/kubeflow).

The task the agent will be learning to perform is to operate a Kuka Robotics arm simulated in the OpenAI Gym Bullet Physics 'KukaBulletEnv-v0' environment. Feel free to [skip to the end](http://localhost:8888/notebooks/kubeflow-rl/apps/agents_ppo/demo.ipynb#Rendering-the-model) to see what this will look like!

### Setup

We need to create a Google Cloud Storage bucket to store job logs as well as a unique subdirectory of that bucket to store logs for this particular run. With the following we first create the GCS bucket then generate the path of a log dir to use in a later step.

Set the variables below to a project and bucket suitable for your use.

In [24]:
# GCP project to use
PROJECT="kubeflow-rl"

# Bucket to use
BUCKET=PROJECT+"-kf"

# K8s cluster to use
CLUSTER="kubeflow-5fc52116"
ZONE="us-east1-d"
NAMESPACE="rl"

# Find the path to the workspace ml-app dir 
import subprocess, os
def find_workspace_root(cwd=None):
    if cwd is None:
        cwd = subprocess.check_output(["pwd"]).strip()
    files = subprocess.check_output(["ls", cwd])
    if "WORKSPACE" in files:
        return cwd
    else:
        cwd = '/'.join(os.path.split(cwd)[:-1])
        return find_workspace_root(cwd)
ML_APP_DIR=os.path.join(find_workspace_root(), "ml-app")

# Needed for launching tensorboard
SECRET_NAME = "gcp-credentials"

**Attention:** You will need GCP credentials to access the cluster and GCP resources.

If you're running this on your local machine you can authenticate for the GCP project you specified above in the usual way (i.e. `gcloud auth login` followed by `gcloud config set project <your-project-name>`; in this case skip the following step.

If you're running this on JupyterLab you can do the following to provide the right credentials:

- Create a service account with the appropriate roles and download the private key
- Use JupyterLab to upload the service account to your pod
- Set the path to your service account in the cell below and then execute it to activate the service account

In [18]:
KEY_FILE="/Users/cb/Downloads/kubeflow-rl-ec0f4f646339.json"
!gcloud auth activate-service-account --key-file={KEY_FILE}

Activated service account credentials for: [cwbeitel-kubeflow-rl@kubeflow-rl.iam.gserviceaccount.com]


In [19]:
!gsutil mb -p {PROJECT} gs://{BUCKET}

Creating gs://kubeflow-rl-kf/...
AccessDeniedException: 403 cwbeitel-kubeflow-rl@kubeflow-rl.iam.gserviceaccount.com does not have storage.buckets.create access to project 991277910492.


In [20]:
!gcloud container clusters --project={PROJECT} --zone={ZONE} get-credentials {CLUSTER}

Fetching cluster endpoint and auth data.
ERROR: (gcloud.container.clusters.get-credentials) ResponseError: code=403, message=Required "container.clusters.get" permission for "projects/kubeflow-rl/zones/us-east1-d/clusters/kubeflow-5fc52116".


In [3]:
!kubectl create namespace {NAMESPACE}

Error from server (AlreadyExists): namespaces "rl" already exists


Download and install ksonnet if needed

In [11]:
!if ! [[ $(which ks) ]]; then mkdir -p ${HOME}/bin && curl -L -o ${HOME}/bin/ks "https://github.com/ksonnet/ksonnet/releases/download/v0.8.0/ks-linux-amd64" && chmod a+rx ${HOME}/bin/ks; fi

If running on GCP (or possibly another Cloud) you probably need to create a key with credentials to use for your job

In [57]:
SECRET_FILE_NAME="secret.json"
!kubectl create -n {NAMESPACE} secret generic {SECRET_NAME} --from-file={SECRET_FILE_NAME}={KEY_FILE}

secret "gcp-credentials" created


### Training

The objective of the training phase is to learn the parameterization of our model that confers a high level of performance on the provided task. Here we'll launch and monitor a job.

#### Launching the TFJob

We'll use [ksonnet](https://ksonnet.io/) to parameterize and apply a TFJob configuration (i.e. run a job). Here you can change the image to be a custom job image, such as one built and deployed with build.sh, or use the one provided here if you only want to change parameters. Below we'll display the templated job YAML for reference.

In [4]:
# Check your cluster and see if that matches one of the existing ksonnet environments
# You want the kubernetes master server to be the same as the server listed for the ks environment
!kubectl cluster-info
!ks env list

Kubernetes master is running at https://35.185.119.177
GLBCDefaultBackend is running at https://35.185.119.177/api/v1/namespaces/kube-system/services/default-http-backend/proxy
Heapster is running at https://35.185.119.177/api/v1/namespaces/kube-system/services/heapster/proxy
KubeDNS is running at https://35.185.119.177/api/v1/namespaces/kube-system/services/kube-dns/proxy
kubernetes-dashboard is running at https://35.185.119.177/api/v1/namespaces/kube-system/services/kubernetes-dashboard/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.
ERROR lstat /Users/cb/environments: no such file or directory


In [6]:
import datetime
import uuid
import os

os.chdir(ML_APP_DIR)

HPARAM_SET="pybullet-kuka"

now=datetime.datetime.now()

JOB_SALT=now.strftime("%m%d-%H%M") + "-" + uuid.uuid4().hex[0:4]
TRAIN_JOB_NAME=HPARAM_SET + "-" + JOB_SALT
LOG_DIR="gs://{0}/jobs/{1}".format(BUCKET, TRAIN_JOB_NAME)

IMAGE="gcr.io/kubeflow-rl/agents:agents-0202-1020-01ec"

!ks param set agents env "KukaBulletEnv-v0"

!ks param set agents run_mode train
!ks param set agents gcp_project kubeflow-rl
!ks param set agents num_cpu 31
!ks param set agents num_agents 30
!ks param set agents sync_replicas False
!ks param set agents steps 4e7
!ks param set agents update_every 30
!ks param set agents max_length 1000
!ks param set agents eval_episodes 25

# Trigger an async render job every 10 minutes
!ks param set agents render_secs 600

!ks param set agents algorithm "agents.algorithms.ppo.ppo.PPO"
!ks param set agents network "agents.scripts.networks.feed_forward_gaussian"

!ks param set agents job_tag {JOB_SALT}
!ks param set agents logdir {LOG_DIR}
!ks param set agents name {TRAIN_JOB_NAME}
!ks param set agents image {IMAGE}
!ks show default -c agents

!ks apply gke -c agents

INFO  Parameter 'env' successfully set to '"KukaBulletEnv-v0"' for component 'agents'
INFO  Parameter 'run_mode' successfully set to '"train"' for component 'agents'
INFO  Parameter 'gcp_project' successfully set to '"kubeflow-rl"' for component 'agents'
INFO  Parameter 'num_cpu' successfully set to '31' for component 'agents'
INFO  Parameter 'num_agents' successfully set to '30' for component 'agents'
INFO  Parameter 'sync_replicas' successfully set to '"False"' for component 'agents'
INFO  Parameter 'steps' successfully set to '4e7' for component 'agents'
INFO  Parameter 'update_every' successfully set to '30' for component 'agents'
INFO  Parameter 'max_length' successfully set to '1000' for component 'agents'
INFO  Parameter 'eval_episodes' successfully set to '25' for component 'agents'
INFO  Parameter 'render_secs' successfully set to '600' for component 'agents'
INFO  Parameter 'algorithm' successfully set to '"agents.algorithms.ppo.ppo.PPO"' for component 'agents'
INFO  Paramete

Now we can list tfjobs and see that a job has been created.

In [7]:
!kubectl get tfjobs -n {NAMESPACE} -o yaml {TRAIN_JOB_NAME}

apiVersion: tensorflow.org/v1alpha1
kind: TfJob
metadata:
  clusterName: ""
  creationTimestamp: 2018-02-02T18:53:36Z
  generation: 0
  name: pybullet-kuka-0202-1053-3a98
  namespace: rl
  resourceVersion: "4043397"
  selfLink: /apis/tensorflow.org/v1alpha1/namespaces/rl/tfjobs/pybullet-kuka-0202-1053-3a98
  uid: 5ff5fc69-084a-11e8-b604-42010af00218
spec:
  RuntimeId: zu80
  replicaSpecs:
  - IsDefaultPS: false
    replicas: 1
    template:
      metadata:
        creationTimestamp: null
      spec:
        containers:
        - args:
          - --run_mode=train
          - --logdir=gs://kubeflow-rl-kf/jobs/pybullet-kuka-0202-1053-3a98
          - --hparam_set_id=pybullet_kuka_ff
          - --run_base_tag=0e90193e
          - --sync_replicas=False
          - --num_gpus=0
          - --algorithm=agents.algorithms.ppo.ppo.PPO
          - --num_agents=30
          - --eval_episodes=25
          - --env=KukaBulletEnv-v0
          - --max_length=1000
          - --steps=40000000
        

#### Monitoring training

The IDs, status, and other metadata of pods involved in the training job can be displayed using the following:

In [8]:
!kubectl get pods -n rl --show-all

NAME                                               READY     STATUS        RESTARTS   AGE
pybullet-kuka-0202-1044-fe76-master-v79e-0-79mc2   1/1       Terminating   0          9m
pybullet-kuka-0202-1052-f773-master-qcpn-0-f8brv   1/1       Terminating   0          1m
pybullet-kuka-0202-1053-3a98-master-zu80-0-cdn9s   1/1       Running       0          2s
ubuntu                                             1/1       Running       0          9d


Logs from a specific pod can be displayed with the following (or streamed by adding the --follow flag):

In [9]:
!kubectl -n {NAMESPACE} get pods -o yaml pybullet-kuka-0202-1053-3a98-master-zu80-0-cdn9s

apiVersion: v1
kind: Pod
metadata:
  annotations:
    kubernetes.io/created-by: |
      {"kind":"SerializedReference","apiVersion":"v1","reference":{"kind":"Job","namespace":"rl","name":"pybullet-kuka-0202-1053-3a98-master-zu80-0","uid":"5ffa2649-084a-11e8-b604-42010af00218","apiVersion":"batch","resourceVersion":"4043392"}}
  creationTimestamp: 2018-02-02T18:53:36Z
  generateName: pybullet-kuka-0202-1053-3a98-master-zu80-0-
  labels:
    controller-uid: 5ffa2649-084a-11e8-b604-42010af00218
    job-name: pybullet-kuka-0202-1053-3a98-master-zu80-0
    job_type: MASTER
    runtime_id: zu80
    task_index: "0"
    tensorflow.org: ""
    tf_job_name: pybullet-kuka-0202-1053-3a98
  name: pybullet-kuka-0202-1053-3a98-master-zu80-0-cdn9s
  namespace: rl
  ownerReferences:
  - apiVersion: batch/v1
    blockOwnerDeletion: true
    controller: true
    kind: Job
    name: pybullet-kuka-0202-1053-3a98-master-zu80-0
    uid: 5ffa2649-084a-11e8-b604-42010af00218
  resourceVersion: "4043403"
  selfL

In [10]:
TRAIN_JOB_NAME

'pybullet-kuka-0202-1053-3a98'

Obtain the ID of the master pod and print logs

In [17]:
import subprocess
TRAIN_JOB_NAME="pybullet-kuka-0202-1053-3a98"
master_pod = subprocess.check_output(["kubectl", "-n", NAMESPACE, "get", "pods", "--selector=tf_job_name=" + TRAIN_JOB_NAME,
                                      "-o", "jsonpath='{.items[*].metadata.name}'"]).decode("utf-8")
print(master_pod)
!kubectl logs -n {NAMESPACE} {master_pod}

'pybullet-kuka-0202-1053-3a98-master-zu80-0-cdn9s'
/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
INFO:tensorflow:Start a new run and write summaries and checkpoints to gs://kubeflow-rl-kf/jobs/pybullet-kuka-0202-1053-3a98.
{'algorithm': <class 'agents.algorithms.ppo.ppo.PPO'>,
 'debug': True,
 'discount': 0.995,
 'dump_dependency_versions': False,
 'env': 'KukaBulletEnv-v0',
 'env_processes': True,
 'eval_episodes': 25,
 'hparam_set_id': 'pybullet_kuka_ff',
 'init_logstd': -1,
 'init_mean_factor': 0.1,
 'init_output_factor': 0.1,
 'init_std': 0.35,
 'kl_cutoff_coef': 1000,
 'kl_cutoff_factor': 2,
 'kl_init_penalty': 1,
 'kl_target': 0.01,
 'learning_rate': 0.0001,
 'log_device_placement': False,
 'logdir': 'gs://kubeflow-rl-kf/jobs/

2018-02-02 18:59:14.574057: I tensorflow/core/kernels/logging_ops.cc:79] policy loss: [0.0954871774]
2018-02-02 18:59:14.574331: I tensorflow/core/kernels/logging_ops.cc:79] current penalty: [5.21509401e-06]
2018-02-02 18:59:14.584562: I tensorflow/core/kernels/logging_ops.cc:79] kl change: [0.00536976848]
2018-02-02 18:59:14.584691: I tensorflow/core/kernels/logging_ops.cc:79] decrease penalty [0]
2018-02-02 18:59:17.230531: I tensorflow/core/kernels/logging_ops.cc:79] return and value: [-0.195469692][-0.357547522]
2018-02-02 18:59:17.230651: I tensorflow/core/kernels/logging_ops.cc:79] normalized advantage: [3.81992641e-06]
2018-02-02 18:59:19.758928: I tensorflow/core/kernels/logging_ops.cc:79] value loss: [0.286263466]
2018-02-02 18:59:19.758954: I tensorflow/core/kernels/logging_ops.cc:79] policy loss: [0.115149826]
2018-02-02 18:59:19.759241: I tensorflow/core/kernels/logging_ops.cc:79] current penalty: [3.47672926e-06]
2018-02-02 18:59:19.769215: I tensorflow/core/kernels/loggin

2018-02-02 19:09:55.200745: I tensorflow/core/kernels/logging_ops.cc:79] value loss: [0.267291844]
2018-02-02 19:09:55.200745: I tensorflow/core/kernels/logging_ops.cc:79] policy loss: [0.0852061212]
2018-02-02 19:09:55.205789: I tensorflow/core/kernels/logging_ops.cc:79] current penalty: [2.09618347e-13]
2018-02-02 19:09:55.221462: I tensorflow/core/kernels/logging_ops.cc:79] kl change: [0.0036235291]
2018-02-02 19:09:55.221624: I tensorflow/core/kernels/logging_ops.cc:79] decrease penalty [0]
INFO:tensorflow:
--------------------------------------------------
Phase eval (phase step 650010, global step 1460010).
INFO:tensorflow:
--------------------------------------------------
Phase eval (phase step 650010, global step 1460010).
INFO:tensorflow:
--------------------------------------------------
Phase train (phase step 810000, global step 1485000).
INFO:tensorflow:
--------------------------------------------------
Phase train (phase step 810000, global step 1485000).
2018-02-02 19:

Phase eval (phase step 800010, global step 1790010).
INFO:tensorflow:
--------------------------------------------------
Phase eval (phase step 800010, global step 1790010).
INFO:tensorflow:
--------------------------------------------------
Phase train (phase step 990000, global step 1815000).
INFO:tensorflow:
--------------------------------------------------
Phase train (phase step 990000, global step 1815000).
2018-02-02 19:14:44.118432: I tensorflow/core/kernels/logging_ops.cc:79] return and value: [-0.317059517][-0.939750791]
2018-02-02 19:14:44.118532: I tensorflow/core/kernels/logging_ops.cc:79] normalized advantage: [4.9691138e-05]
2018-02-02 19:14:46.510082: I tensorflow/core/kernels/logging_ops.cc:79] value loss: [0.136948913]
2018-02-02 19:14:46.510088: I tensorflow/core/kernels/logging_ops.cc:79] policy loss: [0.216507345]
2018-02-02 19:14:46.510428: I tensorflow/core/kernels/logging_ops.cc:79] current penalty: [6.30382433e-17]
2018-02-02 19:14:46.520556: I tensorflow/core

2018-02-02 19:30:38.743090: I tensorflow/core/kernels/logging_ops.cc:79] kl change: [0.015271985]
2018-02-02 19:30:38.743234: I tensorflow/core/kernels/logging_ops.cc:79] increase penalty [0]
INFO:tensorflow:
--------------------------------------------------
Phase eval (phase step 1350000, global step 3000000).
INFO:tensorflow:
--------------------------------------------------
Phase eval (phase step 1350000, global step 3000000).
INFO:tensorflow:
--------------------------------------------------
Phase train (phase step 1650020, global step 3025020).
INFO:tensorflow:
--------------------------------------------------
Phase train (phase step 1650020, global step 3025020).
2018-02-02 19:31:01.635995: I tensorflow/core/kernels/logging_ops.cc:79] return and value: [-0.405077934][-0.0922234952]
2018-02-02 19:31:01.636131: I tensorflow/core/kernels/logging_ops.cc:79] normalized advantage: [5.31222358e-05]
2018-02-02 19:31:04.338249: I tensorflow/core/kernels/logging_ops.cc:79] policy loss:

...

Phase eval (phase step 9650010, global step 21260010).
INFO:tensorflow:
--------------------------------------------------
Phase eval (phase step 9650010, global step 21260010).
INFO:tensorflow:
--------------------------------------------------
Phase train (phase step 11610000, global step 21285000).
INFO:tensorflow:
--------------------------------------------------
Phase train (phase step 11610000, global step 21285000).
2018-02-03 00:46:28.837641: I tensorflow/core/kernels/logging_ops.cc:79] return and value: [-0.279120147][-0.205362603]
2018-02-03 00:46:28.837759: I tensorflow/core/kernels/logging_ops.cc:79] normalized advantage: [-2.26572665e-05]
2018-02-03 00:46:31.332936: I tensorflow/core/kernels/logging_ops.cc:79] kl cutoff! [2]
2018-02-03 00:46:31.456660: I tensorflow/core/kernels/logging_ops.cc:79] kl cutoff! [5]
2018-02-03 00:46:31.576370: I tensorflow/core/kernels/logging_ops.cc:79] kl cutoff! [1]
2018-02-03 00:46:31.687612: I tensorflow/core/kernels/logging_ops.cc:79] va

--------------------------------------------------
Phase train (phase step 11850010, global step 21725010).
2018-02-03 00:53:05.104958: I tensorflow/core/kernels/logging_ops.cc:79] return and value: [-0.280502856][-0.210607037]
2018-02-03 00:53:05.105085: I tensorflow/core/kernels/logging_ops.cc:79] normalized advantage: [-2.39898673e-05]
2018-02-03 00:53:07.900181: I tensorflow/core/kernels/logging_ops.cc:79] value loss: [0.000105762527]
2018-02-03 00:53:07.900189: I tensorflow/core/kernels/logging_ops.cc:79] policy loss: [-0.0723978654]
2018-02-03 00:53:07.900556: I tensorflow/core/kernels/logging_ops.cc:79] current penalty: [0]
2018-02-03 00:53:07.916865: I tensorflow/core/kernels/logging_ops.cc:79] kl change: [0.00319240801]
2018-02-03 00:53:07.917093: I tensorflow/core/kernels/logging_ops.cc:79] decrease penalty [0]
2018-02-03 00:53:10.407704: I tensorflow/core/kernels/logging_ops.cc:79] return and value: [-0.280264616][-0.210547656]
2018-02-03 00:53:10.407803: I tensorflow/core/k

2018-02-03 00:58:22.705823: I tensorflow/core/kernels/logging_ops.cc:79] normalized advantage: [-3.16507976e-05]
2018-02-03 00:58:25.460478: I tensorflow/core/kernels/logging_ops.cc:79] kl cutoff! [2]
2018-02-03 00:58:25.590742: I tensorflow/core/kernels/logging_ops.cc:79] kl cutoff! [2]
2018-02-03 00:58:25.685236: I tensorflow/core/kernels/logging_ops.cc:79] value loss: [8.78626e-05]
2018-02-03 00:58:25.685266: I tensorflow/core/kernels/logging_ops.cc:79] policy loss: [-0.119917586]
2018-02-03 00:58:25.685683: I tensorflow/core/kernels/logging_ops.cc:79] current penalty: [0]
2018-02-03 00:58:25.704623: I tensorflow/core/kernels/logging_ops.cc:79] kl change: [0.0174744818]
2018-02-03 00:58:25.704809: I tensorflow/core/kernels/logging_ops.cc:79] increase penalty [0]
2018-02-03 00:58:27.887408: I tensorflow/core/kernels/logging_ops.cc:79] return and value: [-0.285906762][-0.190020472]
2018-02-03 00:58:27.887520: I tensorflow/core/kernels/logging_ops.cc:79] normalized advantage: [-3.91123

--------------------------------------------------
Phase train (phase step 12240000, global step 22440000).
2018-02-03 01:03:53.836754: I tensorflow/core/kernels/logging_ops.cc:79] return and value: [-0.290788919][-0.185660839]
2018-02-03 01:03:53.836906: I tensorflow/core/kernels/logging_ops.cc:79] normalized advantage: [4.16903167e-05]
2018-02-03 01:03:56.973137: I tensorflow/core/kernels/logging_ops.cc:79] value loss: [0.000120710945]
2018-02-03 01:03:56.973178: I tensorflow/core/kernels/logging_ops.cc:79] policy loss: [-0.0992927551]
2018-02-03 01:03:56.977699: I tensorflow/core/kernels/logging_ops.cc:79] current penalty: [0]
2018-02-03 01:03:56.988647: I tensorflow/core/kernels/logging_ops.cc:79] kl change: [0.0059877797]
2018-02-03 01:03:56.988827: I tensorflow/core/kernels/logging_ops.cc:79] decrease penalty [0]
2018-02-03 01:04:00.675435: I tensorflow/core/kernels/logging_ops.cc:79] return and value: [-0.292191595][-0.188124701]
2018-02-03 01:04:00.675566: I tensorflow/core/ker

--------------------------------------------------
Phase eval (phase step 10325010, global step 22745010).
INFO:tensorflow:
--------------------------------------------------
Phase train (phase step 12420000, global step 22770000).
INFO:tensorflow:
--------------------------------------------------
Phase train (phase step 12420000, global step 22770000).
2018-02-03 01:09:23.773733: I tensorflow/core/kernels/logging_ops.cc:79] return and value: [-0.318865389][-0.208427191]
2018-02-03 01:09:23.773845: I tensorflow/core/kernels/logging_ops.cc:79] normalized advantage: [-6.44938154e-06]
2018-02-03 01:09:26.866150: I tensorflow/core/kernels/logging_ops.cc:79] value loss: [0.000354202057]
2018-02-03 01:09:26.866216: I tensorflow/core/kernels/logging_ops.cc:79] policy loss: [-0.0358680785]
2018-02-03 01:09:26.866643: I tensorflow/core/kernels/logging_ops.cc:79] current penalty: [0]
2018-02-03 01:09:26.884976: I tensorflow/core/kernels/logging_ops.cc:79] kl change: [0.01136728]
2018-02-03 01:0

#### Debug

Re-try with last version of agents known to work propperly: 459c4f88ece996eac3489e6e97a6ee0b30bdd6b3

In [56]:
IMAGE="gcr.io/kubeflow-rl/agents:agents-0204-1005-9e57"

HPARAM_SET="pybullet-kuka"
now=datetime.datetime.now()
JOB_SALT=now.strftime("%m%d-%H%M") + "-" + uuid.uuid4().hex[0:4]
TRAIN_JOB_NAME=HPARAM_SET + "-" + JOB_SALT
LOG_DIR="gs://{0}/jobs/{1}".format(BUCKET, TRAIN_JOB_NAME)

!ks param set agents algorithm "agents.ppo.PPOAlgorithm"
!ks param set agents run_mode train
!ks param set agents job_tag {JOB_SALT}
!ks param set agents logdir {LOG_DIR}
!ks param set agents name {TRAIN_JOB_NAME}
!ks param set agents image {IMAGE}
!ks apply gke -c agents

INFO  Parameter 'algorithm' successfully set to '"agents.ppo.PPOAlgorithm"' for component 'agents'
INFO  Parameter 'run_mode' successfully set to '"train"' for component 'agents'
INFO  Parameter 'job_tag' successfully set to '"0204-1028-bc38"' for component 'agents'
INFO  Parameter 'logdir' successfully set to '"gs://kubeflow-rl-kf/jobs/pybullet-kuka-0204-1028-bc38"' for component 'agents'
INFO  Parameter 'name' successfully set to '"pybullet-kuka-0204-1028-bc38"' for component 'agents'
INFO  Parameter 'image' successfully set to '"gcr.io/kubeflow-rl/agents:agents-0204-1005-9e57"' for component 'agents'
INFO  Updating tfjobs rl.pybullet-kuka-0204-1028-bc38
INFO  Creating non-existent tfjobs rl.pybullet-kuka-0204-1028-bc38


In [48]:
TRAIN_JOB_NAME

'pybullet-kuka-0204-0957-e6e7'

In [55]:
import subprocess
master_pod = subprocess.check_output(["kubectl", "-n", NAMESPACE, "get", "pods", "--selector=tf_job_name=" + TRAIN_JOB_NAME,
                                      "-o", "jsonpath='{.items[*].metadata.name}'"]).decode("utf-8")
print(master_pod)
!kubectl logs -n {NAMESPACE} {master_pod}

'pybullet-kuka-0204-0957-e6e7-master-6vuo-0-fnlnk pybullet-kuka-0204-0957-e6e7-master-vy2k-0-xn8t2'
Error from server (NotFound): pods "pybullet-kuka-0204-0957-e6e7-master-6vuo-0-fnlnk pybullet-kuka-0204-0957-e6e7-master-vy2k-0-xn8t2" not found


#### Launching tensorboard

In [57]:
HPARAM_SET="tboard"
now=datetime.datetime.now()
JOB_SALT=now.strftime("%m%d-%H%M") + "-" + uuid.uuid4().hex[0:4]
TBOARD_JOB_NAME=HPARAM_SET + "-" + JOB_SALT
# LOG_DIR="gs://kubeflow-rl-kf/jobs/pybullet-kuka-0202-1053-3a98"
SECRET_NAME="gcp-credentials"
SECRET_FILE_NAME="secret.json"
NAMESPACE="rl"

#!ks param set tensorboard name {TRAIN_JOB_NAME}
!ks param set tensorboard name {TBOARD_JOB_NAME}
!ks param set tensorboard namespace {NAMESPACE}
!ks param set tensorboard log_dir {LOG_DIR}
!ks param set tensorboard secret {SECRET_NAME}
!ks param set tensorboard secret_file_name {SECRET_FILE_NAME}
!ks show default -c tensorboard

!ks apply gke -c tensorboard

INFO  Parameter 'name' successfully set to '"tboard-0204-1029-9727"' for component 'tensorboard'
INFO  Parameter 'namespace' successfully set to '"rl"' for component 'tensorboard'
INFO  Parameter 'log_dir' successfully set to '"gs://kubeflow-rl-kf/jobs/pybullet-kuka-0204-1028-bc38"' for component 'tensorboard'
INFO  Parameter 'secret' successfully set to '"gcp-credentials"' for component 'tensorboard'
INFO  Parameter 'secret_file_name' successfully set to '"secret.json"' for component 'tensorboard'
---
apiVersion: apps/v1beta1
kind: Deployment
metadata:
  name: tboard-0204-1029-9727-tb
  namespace: rl
spec:
  replicas: 1
  template:
    metadata:
      labels:
        app: tensorboard
        tb-job: tboard-0204-1029-9727
      name: tboard-0204-1029-9727
      namespace: rl
    spec:
      containers:
      - command:
        - /usr/local/bin/tensorboard
        - --logdir=gs://kubeflow-rl-kf/jobs/pybullet-kuka-0204-1028-bc38
        - --port=80
        env:
        - name: GOOGLE_APP

### Connecting to Tensorboard

To connect to tensorboard use kubectl proxy and then access it and the url given by the URL returned by evaluating the next cell

In [58]:
PROXY_PORT=8001
# url=("http://127.0.0.1:{proxy_port}/api/v1/proxy/namespaces/{namespace}/services/{service_name}:80/".format(
#     proxy_port=PROXY_PORT, namespace=NAMESPACE, service_name=TRAIN_JOB_NAME + "-tb"))
url=("http://127.0.0.1:{proxy_port}/api/v1/proxy/namespaces/{namespace}/services/{service_name}:80/".format(
    proxy_port=PROXY_PORT, namespace=NAMESPACE, service_name=TBOARD_JOB_NAME + "-tb"))
print(url)

http://127.0.0.1:8001/api/v1/proxy/namespaces/rl/services/tboard-0204-1029-9727-tb:80/


### Deleting jobs

In [34]:
!kubectl delete tfjobs -n {NAMESPACE} {TRAIN_JOB_NAME}

tfjob "pybullet-kuka-0123-1000-3ad9" deleted


### Rendering the model

#### Initiating a rendering job directly

Launching a rendering job is as simple as the following:

In [59]:
import datetime
import uuid
import os

os.chdir(ML_APP_DIR)

now=datetime.datetime.now()

JOB_SALT=now.strftime("%m%d-%H%M") + "-" + uuid.uuid4().hex[0:4]
RENDER_JOB_NAME="render-" + JOB_SALT

# IMAGE="gcr.io/kubeflow-rl/agents:agents-0202-1020-01ec"
# DEBUG:
IMAGE="gcr.io/kubeflow-rl/agents:agents-0204-1005-9e57"

!ks param set agents name {RENDER_JOB_NAME}

# Note for rendering our resource needs are much smaller
!ks param set agents num_cpu 4

# By default the logdir for the last submitted job will be rendered. To render for
# an arbitrary logdir, specify it here.
# !ks param set agents logdir gs://kubeflow-rl-kf/jobs/pybullet-kuka-0202-1053-3a98

!ks param set agents run_mode render
!ks apply gke -c agents

INFO  Parameter 'name' successfully set to '"render-0204-1029-cd74"' for component 'agents'
INFO  Parameter 'num_cpu' successfully set to '4' for component 'agents'
INFO  Parameter 'run_mode' successfully set to '"render"' for component 'agents'
INFO  Updating tfjobs rl.render-0204-1029-cd74
INFO  Creating non-existent tfjobs rl.render-0204-1029-cd74


In [54]:
%%bash
kubectl get pods -n rl --show-all

NAME                                               READY     STATUS      RESTARTS   AGE
pybullet-kuka-0202-1053-3a98-master-zu80-0-cdn9s   0/1       Completed   0          1d
pybullet-kuka-0204-0957-e6e7-master-6vuo-0-fnlnk   1/1       Running     0          9m
pybullet-kuka-0204-0957-e6e7-master-vy2k-0-xn8t2   1/1       Running     0          10m
render-0203-0816-542f-master-thdv-0-qdcnk          0/1       Completed   0          1d
render-0204-0959-d84c-master-mmvp-0-257ls          1/1       Running     0          8m
render-0204-1006-2883-master-qe7m-0-s2kql          1/1       Running     0          1m
tboard-0204-0911-f303-tb-688bbb9cbb-kw6rm          1/1       Running     0          56m
ubuntu                                             1/1       Running     0          11d


In [60]:
import subprocess
master_pod = subprocess.check_output(["kubectl", "-n", NAMESPACE, "get", "pods", "--selector=tf_job_name=" + RENDER_JOB_NAME,
                                      "-o", "jsonpath='{.items[*].metadata.name}'"]).decode("utf-8")
print(master_pod)
!kubectl logs -n {NAMESPACE} {master_pod}

'render-0204-1029-cd74-master-q4z0-0-dr9hx'
/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
INFO:tensorflow:Resume run and write summaries and checkpoints to gs://kubeflow-rl-kf/jobs/pybullet-kuka-0204-1028-bc38.
INFO:tensorflow:Resume run and write summaries and checkpoints to gs://kubeflow-rl-kf/jobs/pybullet-kuka-0204-1028-bc38.
{'algorithm': <class 'agents.ppo.algorithm.PPOAlgorithm'>,
 'debug': True,
 'discount': 0.995,
 'dump_dependency_versions': False,
 'env': 'KukaBulletEnv-v0',
 'env_processes': True,
 'eval_episodes': 25,
 'hparam_set_id': 'pybullet_kuka_ff',
 'init_logstd': -1,
 'init_mean_factor': 0.1,
 'init_output_factor': 0.1,
 'init_std': 0.35,
 'kl_cutoff_coef': 1000,
 'kl_cutoff_factor': 2,
 'kl_init_penalty': 1,
 '

List the contents of the render subdir that was created by the render job (when finished):

In [61]:
!gsutil ls {LOG_DIR}/render

gs://kubeflow-rl-kf/jobs/pybullet-kuka-0204-1028-bc38/render/0204-1829-3a10/


In [ ]:
!gsutil ls {LOG_DIR}/render/

In [62]:
!mkdir -p /tmp/agents-render
!gsutil cp `gsutil ls {LOG_DIR}/render/0204-1829-3a10/ | grep mp4 | head -n1` /tmp/agents-render/render.mp4

Copying gs://kubeflow-rl-kf/jobs/pybullet-kuka-0204-1028-bc38/render/0204-1829-3a10/openaigym.video.0.44.video000000.mp4...
- [1 files][197.0 KiB/197.0 KiB]                                                
Operation completed over 1 objects/197.0 KiB.                                    


#### Inspecting the result

When the job is complete there will be a subdirectory of the log dir named "render" with a number of short videos of episodes of the agent performing the grasping task. Here's an example of what one of those looks like in a well-trained model.

In [63]:
import io
import base64
from IPython.display import HTML

mp4_path = '/tmp/agents-render/render.mp4'

video = io.open(mp4_path, 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))

### Great job! 🎉🎉🎉

If this is your first time working with these technologies you might be interested in some suggestions of good next steps. Here are some ideas:
- Try training with some other learning environments (from the ID fields [here](https://github.com/bulletphysics/bullet3/blob/master/examples/pybullet/gym/pybullet_envs/__init__.py)) and tweet your results! E.g.
    - RacecarBulletEnv-v0
    - MinitaurBulletDuckEnv-v0
    - HalfCheetahBulletEnv-v0
- Take a shot at implementing your own gym learning environment and repeat the above.